## IMDB 영화평 감성분석(이진 분류)

In [1]:
%cd /content/drive/MyDrive/multi/0428

/content/drive/MyDrive/multi/0428


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd

In [4]:
train = pd.read_csv('./train/trainData.tsv', sep='\t', quoting=3)
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
test = pd.read_csv('./test/testData.tsv', sep='\t', quoting=3)
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      25000 non-null  object
 1   review  25000 non-null  object
dtypes: object(2)
memory usage: 390.8+ KB


In [8]:
np.unique(train['sentiment'].values, return_counts=True)

(array([0, 1]), array([12500, 12500]))

In [9]:
train.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

- 텍스트 전처리

In [10]:
# <br /> 태그는 공백으로 변환
train['review'] = train['review'].str.replace('<br />', ' ')

In [11]:
# 구둣점, 숫자 제거
import re
train['review'] = train['review'].str.replace('[^a-zA-Z]', ' ').str.strip()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
train['review'][0][:1000]

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

- Train/Test dataset 분석

In [13]:
from sklearn.model_selection import train_test_split
X = train.review
y = train.sentiment

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=2022 
)

In [14]:
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

- CountVectorizer로 변환

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

cvect = CountVectorizer(stop_words='english')
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_train_cv.shape

(18750, 65213)

In [16]:
# test dataset은 train dataset을 변환한 Vectorizer로 변환해야 함.
X_test_cv = cvect.transform(X_test)
X_test_cv.shape

(6250, 65213)

- 분류기: Naive Bayes

In [17]:
from sklearn.naive_bayes import MultinomialNB as MNB

mnb = MNB()

In [18]:
mnb.fit(X_train_cv, y_train)

MultinomialNB()

In [19]:
mnb.score(X_test_cv, y_test)

0.85504

- ngram_range : (1,2)

In [20]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv = cvect2.transform(X_train)
X_train_cv.shape

(18750, 1386558)

In [22]:
X_test_cv = cvect2.transform(X_test)
X_test_cv.shape

(6250, 1386558)

In [23]:
mnb = MNB()
mnb.fit(X_train_cv, y_train)
mnb.score(X_test_cv, y_test)

0.8672

- 모델 저장하고 불러오기

In [24]:
import joblib

joblib.dump(cvect2, 'imdb_cvect12.pkl')
joblib.dump(mnb, 'imdb_nb2.pkl')

['imdb_nb2.pkl']

In [25]:
!ls -l

total 81645
-rw------- 1 root root    12949 Apr 28 01:20 01_BagofWord.ipynb
-rw------- 1 root root    26266 Apr 28 02:41 11_IMDB.ipynb
-rw------- 1 root root 39184702 Apr 28 02:40 imdb_cvect12.pkl
-rw------- 1 root root 44370573 Apr 28 02:40 imdb_nb2.pkl
drwx------ 2 root root     4096 Apr 28 01:28 test
drwx------ 3 root root     4096 Apr 28 01:31 train


In [26]:
# 모델 불러오기
new_cvect = joblib.load('imdb_cvect12.pkl')
new_nb = joblib.load('imdb_nb2.pkl')

In [27]:
review = '''
Robert P is a terrible Batman. 
Absolutely no Bat-charisma at all.
The Riddler character is a complete bore! I hated all of this film.
I certainly do not understand the good reviews.
It's not close to the Dark Knight caliber. Please do not make a sequel.
'''

In [28]:
import re
review = re.sub('[^A-Za-z]','',review).strip()

In [29]:
review_cv = new_cvect.transform([review])
review_cv.shape

(1, 1386558)

In [31]:
new_nb.predict_proba(review_cv)

array([[0.5, 0.5]])